# Convolutional Neural Network example

[Original video](https://youtu.be/g6kQl_EFn84)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [ ]:
# Create convolutional neural network (CNN)
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10, size=28):
        super(CNN, self).__init__()
        out_channels=16
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8,
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=out_channels,
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(int(out_channels*(size/4)*(size/4)), num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [ ]:
# Hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 20

load_model = True
filename = 'my_checkpoint.pth.tar'

In [ ]:
# Check if it runs correctly
model = CNN()
x = torch.randn(batch_size, 1, 28, 28)

# Run model on the input and print the shape
print(model(x).shape)

torch.Size([1024, 10])


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Initialize network
model = CNN().to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [ ]:
def save_checkpoint(state, filename=filename):
    print('=> Saving checkpoint')
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print('=> Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_acc = checkpoint['acc']

In [ ]:
if load_model and os.path.exists(filename):
    load_checkpoint(torch.load(filename))
else:
    best_acc = 0

=> Loading checkpoint


In [ ]:
# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data', end='')
    else:
        print('Checking accuracy on test data', end='')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = float(num_correct) / float(num_samples) * 100
        #print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

        model.train()
        return acc

In [ ]:
# Train network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    
    mean_loss = sum(losses)/len(losses)
    acc = check_accuracy(test_loader, model)
    print(f'\rLoss at epoch {epoch} is {mean_loss:.5f}. Accuracy is {acc:.2f}')

    if best_acc < acc:
        best_acc = acc
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'acc': best_acc}
        save_checkpoint(checkpoint)

Loss at epoch 0 is 0.10423. Accuracy is 97.14
=> Saving checkpoint
Loss at epoch 1 is 0.10258. Accuracy is 97.22
=> Saving checkpoint
Loss at epoch 2 is 0.10093. Accuracy is 97.25
=> Saving checkpoint
Loss at epoch 3 is 0.09954. Accuracy is 97.29
=> Saving checkpoint
Loss at epoch 4 is 0.09776. Accuracy is 97.31
=> Saving checkpoint
Loss at epoch 5 is 0.09634. Accuracy is 97.30
Loss at epoch 6 is 0.09501. Accuracy is 97.31
Loss at epoch 7 is 0.09326. Accuracy is 97.36
=> Saving checkpoint
Loss at epoch 8 is 0.09192. Accuracy is 97.47
=> Saving checkpoint
Loss at epoch 9 is 0.09100. Accuracy is 97.50
=> Saving checkpoint
Loss at epoch 10 is 0.08957. Accuracy is 97.51
=> Saving checkpoint
Loss at epoch 11 is 0.08831. Accuracy is 97.61
=> Saving checkpoint
Loss at epoch 12 is 0.08720. Accuracy is 97.54
Loss at epoch 13 is 0.08636. Accuracy is 97.62
=> Saving checkpoint
Loss at epoch 14 is 0.08495. Accuracy is 97.67
=> Saving checkpoint
Loss at epoch 15 is 0.08405. Accuracy is 97.66
Loss a

In [ ]:
print(f': {check_accuracy(train_loader, model):.2f}')
print(f': {check_accuracy(test_loader, model):.2f}')

Checking accuracy on training data: 97.65
Checking accuracy on test data: 97.75
